In [ ]:
##Funciones para la API

#Debes crear las siguientes funciones para los endpoints que se consumirán en la API,
# recuerden que deben tener un decorador por cada una (@app.get(‘/’)).

# 1--->def PlayTimeGenre( genero : str ): Debe devolver año con mas horas jugadas para dicho género.
#Ejemplo de retorno: {"Año de lanzamiento con más horas jugadas para Género X" : 2013}





In [5]:
import pandas as pd

#Lectura de los archivos CSV
steam_games = pd.read_csv('csv/steam_games.csv')
user_items =pd.read_csv('csv/user_items.csv')

In [2]:
steam_games.info()
user_items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22529 entries, 0 to 22528
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   genero            22529 non-null  object 
 1   app_name          22529 non-null  object 
 2   titulo            22529 non-null  object 
 3   anio_lanzamiento  22495 non-null  float64
 4   precio            22529 non-null  float64
 5   id_contenido      22529 non-null  int64  
 6   desarrollador     22529 non-null  object 
dtypes: float64(2), int64(1), object(4)
memory usage: 1.2+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5094105 entries, 0 to 5094104
Data columns (total 6 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   user_id           object
 1   items_count       int64 
 2   user_url          object
 3   item_id           int64 
 4   item_name         object
 5   playtime_forever  int64 
dtypes: int64(3), object(3)
memory usage: 233.2+ MB


In [3]:
 # Combinar los DataFrames en uno solo usando la función merge
merged_df = pd.merge(steam_games, user_items, left_on='id_contenido', right_on='item_id')

#Convertimos a CSV
merged_df.to_parquet('Archivos_API\\PlayTimeGenre.parquet', index=False)

NameError: name 'pd' is not defined

In [5]:
def PlayTimeGenre(genero):
    
    # Filtrar por género
    merged_df_genero = merged_df[merged_df['genero'].apply(lambda x: genero in x)]
    
    # Si no hay datos para el género, retornar un mensaje de error
    if merged_df_genero.empty:
        return {f"No se encontraron datos para el género {genero}": None}
    
    # Calcular las horas jugadas por año de lanzamiento
    horas_por_anio = merged_df_genero.groupby('anio_lanzamiento')['playtime_forever'].sum()
    
    # Encontrar el año con más horas jugadas
    año_max_horas = horas_por_anio.idxmax()
    
    # Devolver el resultado
    return {f"Año de lanzamiento con más horas jugadas para {genero}": año_max_horas}


In [6]:
# Ejemplo de uso
print(PlayTimeGenre("Action"))

{'Año de lanzamiento con más horas jugadas para Action': 2012.0}


In [1]:
##def UserForGenre( genero : str ): Debe devolver el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año.
##Ejemplo de retorno: {"Usuario con más horas jugadas para Género X" : us213ndjss09sdf, "Horas jugadas":[{Año: 2013, Horas: 203}, {Año: 2012, Horas: 100}, {Año: 2011, Horas: 23}]}

def UserForGenre(genero):
      
    # Filtrar por género
    merged_df_genero = merged_df[merged_df['genero'].apply(lambda x: genero in x)]
    
    # Si no hay datos para el género, retornar un mensaje de error
    if merged_df_genero.empty:
        return {f"No se encontraron datos para el género {genero}": None}
    
    # Encontrar el usuario que acumula más horas jugadas
    user_max_horas = merged_df_genero.groupby('user_id')['playtime_forever'].sum().idxmax()
    
    # Calcular la acumulación de horas jugadas por año
    horas_por_anio = merged_df_genero.groupby('anio_lanzamiento')['playtime_forever'].sum().reset_index()
    horas_por_anio = horas_por_anio.rename(columns={'anio_lanzamiento': 'Año', 'playtime_forever': 'Horas'})
    horas_por_anio = horas_por_anio.to_dict(orient='records')
    
    # Devolver el resultado con el formato solicitado
    return {
        f"Usuario con más horas jugadas para {genero}": user_max_horas,
        "Horas jugadas": horas_por_anio
    }


In [2]:
print(UserForGenre("Action"))

NameError: name 'merged_df' is not defined

In [6]:
#3--->def UsersRecommend( año : int ): 
#Devuelve el top 3 de juegos MÁS recomendados por usuarios para el año dado. 
#(reviews.recommend = True y comentarios positivos/neutrales)

#Ejemplo de retorno: [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}]



user_reviews =pd.read_csv('csv/user_reviews_with_sentiment.csv')
user_reviews.info()

# Combinar DataFrames basado en las columnas id y item_id
merge_df_3= pd.merge(user_reviews, steam_games, left_on='item_id', right_on='id_contenido', how='inner')
merge_df_3.head()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48463 entries, 0 to 48462
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   user_id             48463 non-null  object
 1   posted              48463 non-null  int64 
 2   item_id             48463 non-null  int64 
 3   recommend           48463 non-null  bool  
 4   review              48463 non-null  object
 5   sentiment_analysis  48463 non-null  int64 
dtypes: bool(1), int64(3), object(2)
memory usage: 1.9+ MB


,user_id,posted,item_id,recommend,review,sentiment_analysis,genero,app_name,titulo,anio_lanzamiento,precio,id_contenido,desarrollador
0,76561197970982479,2011,1250,True,Simple yet with great replayability. In my opi...,2,['Action'],Killing Floor,Killing Floor,2009.0,19.99,1250,Tripwire Interactive
1,death-hunter,2015,1250,True,"Amazing, Non-stop action of blowing stuff to b...",2,['Action'],Killing Floor,Killing Floor,2009.0,19.99,1250,Tripwire Interactive
2,DJKamBer,2013,1250,True,"Compared to Left 4 Dead 2, this game REALLY gi...",0,['Action'],Killing Floor,Killing Floor,2009.0,19.99,1250,Tripwire Interactive
3,diego9031,2015,1250,True,Jogo ‚ô•‚ô•‚ô•‚ô•.,1,['Action'],Killing Floor,Killing Floor,2009.0,19.99,1250,Tripwire Interactive
4,76561198081962345,2014,1250,True,cara nas imagens esse jogo da pouco de medo ma...,1,['Action'],Killing Floor,Killing Floor,2009.0,19.99,1250,Tripwire Interactive


In [7]:
# Seleccionar las columnas necesarias
selected_columns = ['app_name', 'posted', 'recommend', 'sentiment_analysis']
df_UsersRecommend = merge_df_3[selected_columns]
df_UsersRecommend.head()

,app_name,posted,recommend,sentiment_analysis
0,Killing Floor,2011,True,2
1,Killing Floor,2015,True,2
2,Killing Floor,2013,True,0
3,Killing Floor,2015,True,1
4,Killing Floor,2014,True,1


In [8]:
#Filtramos los comentarios positivos y neutrales
# Filter rows where sentiment_analysis is not 0 and recommend is True
df_UsersRecommend = merge_df_3[(merge_df_3['sentiment_analysis'] != 0) & (merge_df_3['recommend'] == True)]
df_UsersRecommend.head()


,user_id,posted,item_id,recommend,review,sentiment_analysis,genero,app_name,titulo,anio_lanzamiento,precio,id_contenido,desarrollador
0,76561197970982479,2011,1250,True,Simple yet with great replayability. In my opi...,2,['Action'],Killing Floor,Killing Floor,2009.0,19.99,1250,Tripwire Interactive
1,death-hunter,2015,1250,True,"Amazing, Non-stop action of blowing stuff to b...",2,['Action'],Killing Floor,Killing Floor,2009.0,19.99,1250,Tripwire Interactive
3,diego9031,2015,1250,True,Jogo ‚ô•‚ô•‚ô•‚ô•.,1,['Action'],Killing Floor,Killing Floor,2009.0,19.99,1250,Tripwire Interactive
4,76561198081962345,2014,1250,True,cara nas imagens esse jogo da pouco de medo ma...,1,['Action'],Killing Floor,Killing Floor,2009.0,19.99,1250,Tripwire Interactive
5,GamerFag,2010,1250,True,An Awesome co-op survival shooter! BUY IT!!!,2,['Action'],Killing Floor,Killing Floor,2009.0,19.99,1250,Tripwire Interactive


In [9]:
#Agrupamos por app_name , posted_year y sumamos la cantidad de recomendaciones
df_UsersRecommend = df_UsersRecommend.groupby(['app_name', 'posted']).agg({'recommend': 'sum'}).reset_index()
df_UsersRecommend = df_UsersRecommend.sort_values('recommend',ascending=False)
df_UsersRecommend.head()

,app_name,posted,recommend
2798,Team Fortress 2,2014,1416
622,Counter-Strike: Global Offensive,2015,1317
621,Counter-Strike: Global Offensive,2014,948
2797,Team Fortress 2,2013,748
1223,Garry's Mod,2014,684


In [10]:
#Convertimos a CSV
df_UsersRecommend.to_parquet('Archivos_API\\UsersRecommend.parquet', index=False)

In [16]:
def UsersRecommend(anio):
 
    # Filtrar las recomendaciones para el año dado y recomendaciones positivas/neutrales
    recomendaciones = df_UsersRecommend[df_UsersRecommend['posted'] == anio]

    # Verificar si hay revisiones para el año dado
    if recomendaciones.empty:
        return f"No hay recomendaciones para el año {anio}"

    # Ordenar en orden descendente por la cantidad de recomendaciones
    recomendaciones = recomendaciones.sort_values('recommend', ascending=False)

    # Crear una única línea de resultado
    resultado = {
        "Puesto 1": recomendaciones.iloc[0]['app_name'],
        "Puesto 2": recomendaciones.iloc[1]['app_name'],
        "Puesto 3": recomendaciones.iloc[2]['app_name']
    }

    return resultado

In [17]:
UsersRecommend(2011)

{'Puesto 1': 'Team Fortress 2', 'Puesto 2': 'Portal 2', 'Puesto 3': 'Terraria'}

In [18]:
#4--->def UsersNotRecommend( año : int ): 
#Devuelve el top 3 de juegos MENOS recomendados por usuarios para el año dado. 
#(reviews.recommend = False y comentarios negativos)

In [19]:
#Filtramos los comentarios negativos

df_UsersnotRecommend = merge_df_3[(merge_df_3['sentiment_analysis'] == 0) & (merge_df_3['recommend'] == False)]
df_UsersnotRecommend.head()


,user_id,posted,item_id,recommend,review,sentiment_analysis,genero,app_name,titulo,anio_lanzamiento,precio,id_contenido,desarrollador
102,76561198067664442,2014,1250,False,why i cant play multiplayer ? f10 to cancel co...,0,['Action'],Killing Floor,Killing Floor,2009.0,19.99,1250,Tripwire Interactive
106,76561198016635228,2014,1250,False,"repetitive gameplay, nuff said about it.",0,['Action'],Killing Floor,Killing Floor,2009.0,19.99,1250,Tripwire Interactive
270,76561198067771967,2015,1250,False,"quite a bad game,hardly any online severs",0,['Action'],Killing Floor,Killing Floor,2009.0,19.99,1250,Tripwire Interactive
393,TheDamox,2015,227300,False,It really is a waste of money,0,"['Indie', 'Simulation']",Euro Truck Simulator 2,Euro Truck Simulator 2,2013.0,19.99,227300,SCS Software
523,wolvau,2015,224500,False,Boring,0,"['Indie', 'Simulation', 'Strategy']",Gnomoria,Gnomoria,2016.0,14.99,224500,Robotronic Games


In [20]:
#Agrupamos por app_name , posted_year y sumamos la cantidad de recomendaciones
df_UsersnotRecommend = df_UsersnotRecommend.groupby(['app_name', 'posted']).agg({'recommend': 'sum'}).reset_index()
df_UsersnotRecommend = df_UsersnotRecommend.sort_values('recommend',ascending=False)
df_UsersnotRecommend.head()

,app_name,posted,recommend
0,100% Orange Juice,2015,0
501,Red Orchestra 2: Heroes of Stalingrad with Ris...,2015,0
492,RaceRoom Racing Experience,2015,0
493,Ragnarok Online 2,2014,0
494,Ragnarok Online 2,2015,0


In [21]:
#Convertimos a CSV
df_UsersnotRecommend .to_parquet('Archivos_API\\UsernotRecommend.parquet', index=False)

In [22]:
def UsersNotRecommend(anio):
     

    # Filtrar las recomendaciones para el año dado y recomendaciones positivas/neutrales
    recomendaciones = df_UsersnotRecommend[df_UsersnotRecommend['posted'] == anio]

    # Verificar si hay revisiones para el año dado
    if recomendaciones.empty:
        return f"No hay recomendaciones para el año {anio}"

    # Ordenar en orden descendente por la cantidad de recomendaciones
    recomendaciones = recomendaciones.sort_values('recommend', ascending=False)

    # Crear una única línea de resultado
    resultado = {
        "Puesto 1": recomendaciones.iloc[0]['app_name'],
        "Puesto 2": recomendaciones.iloc[1]['app_name'],
        "Puesto 3": recomendaciones.iloc[2]['app_name']
    }

    return resultado

In [24]:
UsersNotRecommend(2011)

{'Puesto 1': 'Men of War: Vietnam',
 'Puesto 2': 'Team Fortress 2',
 'Puesto 3': 'And Yet It Moves'}

def sentiment_analysis( año : int ): Según el año de lanzamiento, se devuelve una lista con la cantidad de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento.
Ejemplo de retorno: {Negative = 182, Neutral = 120, Positive = 278}

In [25]:
user_reviews_df =pd.read_csv('csv/user_reviews_with_sentiment.csv')
user_reviews.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48463 entries, 0 to 48462
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   user_id             48463 non-null  object
 1   posted              48463 non-null  int64 
 2   item_id             48463 non-null  int64 
 3   recommend           48463 non-null  bool  
 4   review              48463 non-null  object
 5   sentiment_analysis  48463 non-null  int64 
dtypes: bool(1), int64(3), object(2)
memory usage: 1.9+ MB


In [26]:
#Convertimos a CSV
user_reviews_df .to_parquet('Archivos_API\\UsernotRecommend.parquet', index=False)

In [27]:
def sentiment_analysis(año: int):
    # Filtrar el DataFrame de reseñas por el año dado
    filtered_reviews = user_reviews_df[user_reviews['posted'] == año]
    
    # Contar la cantidad de registros para cada categoría de análisis de sentimiento
    sentiment_counts = filtered_reviews['sentiment_analysis'].value_counts()
    
    # Crear un diccionario para almacenar los resultados
    result = {
        'Negative': sentiment_counts.get(0, 0),  # 0 para negativo
        'Neutral': sentiment_counts.get(1, 0),   # 1 para neutral
        'Positive': sentiment_counts.get(2, 0)   # 2 para positivo
    }
    
    return result



In [28]:
# Ejemplo de uso:
año_lanzamiento = 2012
resultados = sentiment_analysis(año_lanzamiento)  # Suponiendo que 'merge_df_3' es tu DataFrame de reseñas
print(resultados)

{'Negative': 139, 'Neutral': 231, 'Positive': 831}


In [60]:
#Convertimos a CSV
user_reviews_df .to_parquet('Archivos_API\\sentiment_analysis.parquet', index=False)